## Aspect Based Sentiment Analysis

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import torch.nn.functional as F
import gc

In [3]:
# Before running the code, make sure cleaned data is available and named 'Black Myth WuKong Reviews.csv'

In [4]:
reviews_df = pd.read_csv('Black Myth WuKong Reviews.csv')
reviews_df["review"].head()

0    Amazing game. The detail put into side bosses ...
1    better than i thought it would be. would be a ...
2      try this game one time you will never forgot it
3    great game. tons of bosses. great art style. g...
4    I haven't played a game all the way through in...
Name: review, dtype: object

In [6]:
# Create the Term-Document Matrix (or TF-IDF matrix)
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = 0.95, min_df = 2)
tfidf_matrix = vectorizer.fit_transform(reviews_df['review'])
feature_names = vectorizer.get_feature_names_out()

In [8]:
# Create seed words for words relevant to aspects
aspect_seed_words = {
    'gameplay': [
        'gameplay', 'controls', 'mechanics', 'playing', 'combat', 'puzzle', 'challenge',
        'actions', 'movement', 'abilities', 'skills', 'weapons', 'armor', 'crafting',
        'progression', 'exploration', 'loot', 'flow', 'pace', 'loop',
        'strategy', 'tactics', 'difficulty', 'grind', 'replayability', 'fun', 'satisfying',
        'grinding', 'perks', 'upgrades', 'economy', 'resources', 'interaction',
        'decision', 'choice', 'sandbox', 'survival'],
    'graphics': [
        'graphics', 'visuals', 'art', 'stunning', 'beautiful', 'fidelity', 'textures',
        'models', 'lighting', 'animations', 'style', 'aesthetic', 'design', 'look',
        'appearance', 'vibrant', 'detailed', 'environments', 'shader', 'particles'],
    'story': [
        'story', 'narrative', 'plot', 'characters', 'lore', 'dialogue', 'writing', 'twist',
        'ending', 'protagonist', 'antagonist', 'cutscenes', 'meaningful',
        'emotional', 'depth', 'background', 'quests', 'backstory', 'theme'],
    'audio': [
        'sound', 'music', 'audio', 'voice', 'immersive', 'soundtrack', 'effects',
        'score', 'ambience', 'voiceovers', 'ear', 'mix'],
    'performance': [
        'performance', 'fps', 'lag', 'runs', 'smooth', 'crashes', 'stutter', 'frames',
        'optimization', 'stable', 'glitchy', 'loading', 'buggy', 'issues',
        'technical', 'stability', 'hardware', 'specifications', 'overheating', 'slowdown'],
    'multiplayer': [
        'multiplayer', 'online', 'pvp', 'community', 'versus', 'competitive', 'team', 'social', 'friends',
        'clan', 'guild', 'latency', 'ping', 'leaderboards'],
    'level_design': [
        'level', 'design', 'maps', 'environments', 'layout', 'boss', 'unlock', 'achievement',
        'explore', 'area', 'zone', 'dungeon', 'world', 'pacing', 'structure', 'flow',
        'secrets', 'paths', 'traps', 'obstacles', 'variety', 'verticality', 'landmarks',
        'sprawling', 'linear', 'hub', 'missions', 'objectives', 'quests',
        'progression', 'landmarks', 'scale', 'architecture', 'space', 'respawn', 'balance', 'fairness'],
    'bugs': [
        'bugs', 'flaw', 'buggy', 'glitches', 'error', 'crashes', 'broken', 'unplayable',
        'issues', 'problems', 'fixes', 'patch', 'update', 'defects', 'stability', 'corrupt']
}
n_aspects = len(aspect_seed_words)

In [10]:
# Create the guiding matrix H
H_init = np.zeros((n_aspects, len(feature_names)))
vocab = {word: i for i, word in enumerate(feature_names)}

# Strength here encourages the NMF to associate seed words strongly with their respective topics
seed_strength = 10.0
default_strength = 0.1

for i, (aspect, seeds) in enumerate(aspect_seed_words.items()):
    for seed_word in seeds:
        if seed_word in vocab:
            H_init[i, vocab[seed_word]] = seed_strength
        else:
            print(f"Warning: Seed word '{seed_word}' for aspect '{aspect}' not in TF-IDF vocabulary.")

In [12]:
# Initialize NMF with the guiding matrix
nmf_model = NMF(
    n_components = n_aspects,
    init = 'custom',
    solver = 'mu',
    beta_loss = 'frobenius',
    max_iter = 10000,
    random_state = 42,
    alpha_W = 0.0,
    alpha_H = 0.0
)

In [14]:
# Create random W or feature matrix to begin
W_init = np.random.rand(tfidf_matrix.shape[0], n_aspects) * 0.1

W = nmf_model.fit_transform(tfidf_matrix, W = W_init, H = H_init)
H = nmf_model.components_
learned_aspect_names = list(aspect_seed_words.keys())

In [16]:
# Extract top aspects for the set categories using this function
def get_top_terms_for_aspects(seed_words_for_category,
                              H_matrix,
                              feature_names_list,
                              all_learned_component_names,
                              n_top = 5):
    """
    Function to detect the top aspects/terms for the set categories.

    Args:
        seed_words_for_category: the dictionary containing seed words
        H_matrix: guiding matrix
        feature_names_list: list of names of the features extracted during the vectorization of reviews
        all_learned_component_names: list of seed words provided in the guided matrix
        n_top: number of top terms to extract

    Returns: 
        top_terms: dictonary of the top terms for each aspect/category
    """
    top_terms = {}
    for i, component in enumerate(H_matrix):
        top_indices = component.argsort()[-n_top:][::-1]
        top_terms[all_learned_component_names[i]] = [feature_names_list[ind] for ind in top_indices]
    return top_terms

# Run the function to get top terms
aspect_top_terms = get_top_terms_for_aspects(aspect_seed_words, H, feature_names, learned_aspect_names)

In [18]:
# Print out the aspects
for aspect, terms in aspect_top_terms.items():
    print(f"Top terms for {aspect}: {terms}")

Top terms for gameplay: ['fun', 'combat', 'gameplay', 'playing', 'mechanics']
Top terms for graphics: ['graphics', 'beautiful', 'stunning', 'art', 'visuals']
Top terms for story: ['story', 'lore', 'characters', 'background', 'ending']
Top terms for audio: ['music', 'sound', 'immersive', 'voice', 'effects']
Top terms for performance: ['smooth', 'performance', 'issues', 'runs', 'fps']
Top terms for multiplayer: ['friends', 'social', 'team', 'multiplayer', 'community']
Top terms for level_design: ['boss', 'design', 'level', 'world', 'linear']
Top terms for bugs: ['crashes', 'bugs', 'issues', 'update', 'patch']


In [19]:
# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

/Users/blairlee/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use mps:0


In [20]:
# Detect machine capabilities
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")
absa_model.to(device)

Using device: mps


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [23]:
def analyze_review_by_category(review_text, aspect_top_terms_map, tokenizer, model, current_device, internal_batch_size = 16, tokenizer_max_length = 512):
    """
    Analyzes a single review to determine sentiment for predefined aspect categories
    based on the guided NMF top terms.

    Args:
        review_text: The Steam game review
        aspect_top_terms_map: Dictionary where keys are aspect categories and values 
                                     are lists of top NMF terms for that category.
                                     Example: {'gameplay': ['mechanics', 'controls'], ...}
        tokenizer: The ABSA tokenizer.
        model: The ABSA model.
        current_device: The device to run the model on ('mps', 'cuda', or 'cpu').
        internal_batch_size: How many (sentence, term) pairs to process at once by the model.

    Returns:
        pd.Series: A Series with aspect categories as index and predicted sentiment
                   ('positive', 'negative', or 'neutral' if terms are found, else None).
    """
    model_inference_inputs = []

    for aspect_category_name, nmf_terms in aspect_top_terms_map.items():
        for nmf_term in nmf_terms:
            if nmf_term.lower() in review_text.lower():
                model_inference_inputs.append((review_text, nmf_term, aspect_category_name))

    if not model_inference_inputs:
        return pd.Series({category: None for category in aspect_top_terms_map.keys()}, dtype = object)

    all_term_sentiments_data = []

    for i in range(0, len(model_inference_inputs), internal_batch_size):
        current_batch_segment = model_inference_inputs[i:i+internal_batch_size]

        sentences_for_model_batch = [item[0] for item in current_batch_segment]
        terms_for_model_batch = [item[1] for item in current_batch_segment]
        original_categories_in_batch = [item[2] for item in current_batch_segment]

        encoded_batch = tokenizer(
            sentences_for_model_batch,
            terms_for_model_batch,
            return_tensors = "pt",
            padding = True,
            truncation = True,
            max_length = tokenizer_max_length
        )
        encoded_batch = {k: v.to(current_device) for k, v in encoded_batch.items()}

        with torch.no_grad():
            outputs = model(**encoded_batch)
            # Probabilities for [negative, neutral, positive]
            probabilities_batch = F.softmax(outputs.logits, dim = 1).cpu().numpy()

        for j in range(len(probabilities_batch)):
            all_term_sentiments_data.append({
                "original_category": original_categories_in_batch[j],
                "term_sentiment_probs": probabilities_batch[j]
            })

    # Aggregate sentiments for each original aspect category
    final_category_sentiments = {}
    for category_name_key in aspect_top_terms_map.keys():
        probs_for_this_category = [
            data["term_sentiment_probs"] for data in all_term_sentiments_data
            if data["original_category"] == category_name_key
        ]

        if not probs_for_this_category:
            final_category_sentiments[category_name_key] = None # No terms for this category found/analyzed
            continue

        # Average the probabilities across all found terms for this category
        avg_probs_for_category = np.mean(np.array(probs_for_this_category), axis = 0)

        neg_score = avg_probs_for_category[0]
        neutral_score = avg_probs_for_category[1]
        pos_score = avg_probs_for_category[2]

        # Determine final positive/negative prediction
        if pos_score > neg_score:
            final_sentiment = "positive"
        elif neg_score > pos_score:
            final_sentiment = "negative"
        else:
            final_sentiment_idx = np.argmax(avg_probs_for_category)
            class_labels = ["negative", "neutral", "positive"]
            final_sentiment = class_labels[final_sentiment_idx]

        final_category_sentiments[category_name_key] = final_sentiment

    return pd.Series(final_category_sentiments, dtype = object)

In [25]:
# Parameters that can be changed depending on computer capacity/power
chunk_size = 5
internal_model_batch_size = 4
tokenizer_max_len = 256

all_results_list = []

print(f"Using device: {device}")
print(f"DataFrame chunk_size: {chunk_size}")
print(f"Internal model batch_size: {internal_model_batch_size}")
print(f"Tokenizer max_length: {tokenizer_max_len}")

# For loop to process sentiment in chunks
for i in range(0, len(reviews_df), chunk_size):
    review_chunk = reviews_df['review'][i:i + chunk_size]

    current_chunk_results = []
    for review_text in review_chunk: # Process one review at a time from the small chunk
        sentiments = analyze_review_by_category(
            review_text,
            aspect_top_terms,
            absa_tokenizer,
            absa_model,
            device,
            internal_batch_size = internal_model_batch_size
        )
        current_chunk_results.append(sentiments)

    # Convert list of Series to DataFrame for this chunk
    if current_chunk_results:
        chunk_sentiments_df = pd.DataFrame(current_chunk_results)
        all_results_list.append(chunk_sentiments_df)
    
    # Try to free memory after each major chunk
    if device.type == 'mps':
        torch.mps.empty_cache()
    gc.collect()

    print(f"Processed chunk {i//chunk_size + 1} of {(len(reviews_df)-1)//chunk_size + 1}")
    
if all_results_list:
    df_final_aspect_sentiments = pd.concat(all_results_list)
else:
    empty_cols = list(aspect_top_terms.keys())
    df_final_aspect_sentiments = pd.DataFrame(columns = empty_cols)

df_final_aspect_sentiments.head()

Using device: mps
DataFrame chunk_size: 5
Internal model batch_size: 4
Tokenizer max_length: 256
Processed chunk 1 of 7861
Processed chunk 2 of 7861
Processed chunk 3 of 7861
Processed chunk 4 of 7861
Processed chunk 5 of 7861
Processed chunk 6 of 7861
Processed chunk 7 of 7861
Processed chunk 8 of 7861
Processed chunk 9 of 7861
Processed chunk 10 of 7861
Processed chunk 11 of 7861
Processed chunk 12 of 7861
Processed chunk 13 of 7861
Processed chunk 14 of 7861
Processed chunk 15 of 7861
Processed chunk 16 of 7861
Processed chunk 17 of 7861
Processed chunk 18 of 7861
Processed chunk 19 of 7861
Processed chunk 20 of 7861
Processed chunk 21 of 7861
Processed chunk 22 of 7861
Processed chunk 23 of 7861
Processed chunk 24 of 7861
Processed chunk 25 of 7861
Processed chunk 26 of 7861
Processed chunk 27 of 7861
Processed chunk 28 of 7861
Processed chunk 29 of 7861
Processed chunk 30 of 7861
Processed chunk 31 of 7861
Processed chunk 32 of 7861
Processed chunk 33 of 7861
Processed chunk 34 of

,gameplay,graphics,story,audio,performance,multiplayer,level_design,bugs
0,None,None,positive,None,positive,None,positive,None
1,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None
3,positive,positive,None,None,None,None,positive,None
4,positive,positive,None,None,positive,None,None,positive


In [27]:
# Combine the reviews with the aspects and sentiments
reviews_df = reviews_df.reset_index(drop = True)
df_final_aspect_sentiments = df_final_aspect_sentiments.reset_index(drop = True)
df = pd.concat([reviews_df['review'], df_final_aspect_sentiments], axis = 1)
df.head(100)

,review,gameplay,graphics,story,audio,performance,multiplayer,level_design,bugs
0,Amazing game. The detail put into side bosses ...,None,None,positive,None,positive,None,positive,None
1,better than i thought it would be. would be a ...,None,None,None,None,None,None,None,None
2,try this game one time you will never forgot it,None,None,None,None,None,None,None,None
3,great game. tons of bosses. great art style. g...,positive,positive,None,None,None,None,positive,None
4,I haven't played a game all the way through in...,positive,positive,None,None,positive,None,None,positive
...,...,...,...,...,...,...,...,...,...
95,Cool boss battles,None,None,None,None,None,None,positive,None
96,did not find the game enjoyable.,None,None,None,None,None,None,None,None
97,"very nice game! like it, first it is hard to p...",None,None,None,None,None,None,None,None
98,"10/10 game, bought it at full price, it worth ...",None,None,None,None,None,None,None,None


In [29]:
# Aggregate sentiments for each category
aggregated_category_sentiments = {}

aspect_categories = df_final_aspect_sentiments.columns

for category in aspect_categories:
    # Count the occurrences of each sentiment, dropping NaN/None values
    sentiment_counts = df_final_aspect_sentiments[category].value_counts(dropna = True)
    
    # Calculate the total number of valid sentiment entries for this category
    total_valid_reviews = sentiment_counts.sum()

    if total_valid_reviews == 0:
        aggregated_category_sentiments[category] = 'No data'
        continue

    # Calculate percentages
    positive_pct = sentiment_counts.get('positive', 0) / total_valid_reviews
    negative_pct = sentiment_counts.get('negative', 0) / total_valid_reviews
    neutral_pct = sentiment_counts.get('neutral', 0) / total_valid_reviews
    
    if positive_pct > 0.5:
        aggregated_category_sentiments[category] = 'positive'
    elif negative_pct > 0.5:
        aggregated_category_sentiments[category] = 'negative'
    elif neutral_pct > 0.5:
        aggregated_category_sentiments[category] = 'neutral'
    else:
        # Fallback: If no sentiment has >50%, choose the one with the highest count
        if not sentiment_counts.empty:
            most_frequent_sentiment = sentiment_counts.index[0]
            aggregated_category_sentiments[category] = most_frequent_sentiment
        else:
            aggregated_category_sentiments[category] = 'No data' 

# Convert the results to a Pandas Series
overall_sentiments_summary = pd.Series(aggregated_category_sentiments)

print("\nOverall Sentiment Summary for Each Category:")
print(overall_sentiments_summary)


Overall Sentiment Summary for Each Category:
gameplay        positive
graphics        positive
story           positive
audio           positive
performance     positive
multiplayer     positive
level_design    positive
bugs            negative
dtype: object
